In [11]:
import cv2
import numpy as np
import os
from random import shuffle
from tqdm import tqdm

TRAIN_DIR = 'ml65-data/train/'
TEST_DIR = 'ml65-data/test/'
IMG_SIZE = 50
LR = 1e-3

MODEL_NAME = 'dogsvscat-{}-{}.model'.format(LR, '6conv-basic')

In [12]:
def label_image(img):
    # dog.93.png
    word_label = img.split('.')[-3]
    if word_label == 'cat': return [1, 0]
    elif word_label == 'dog': return [0, 1]

In [13]:
def create_training_data():
    training_data = []
    for img in tqdm(os.listdir(TRAIN_DIR)):
        label = label_image(img)
        path = os.path.join(TRAIN_DIR, img)
        img = cv2.resize(cv2.imread(path, cv2.IMREAD_GRAYSCALE), (IMG_SIZE, IMG_SIZE))

        training_data.append([np.array(img), np.array(label)])
    
    shuffle(training_data)
    np.save('ml65-data/train_data.npy', np.array(training_data, dtype=object))
    return training_data

In [14]:
def process_test_data():
    testing_data = []
    for img in tqdm(os.listdir(TEST_DIR)):
        path = os.path.join(TEST_DIR, img)
        img_num = img.split('.')[0]
        img = cv2.resize(cv2.imread(path, cv2.IMREAD_GRAYSCALE), (IMG_SIZE, IMG_SIZE))
        testing_data.append([np.array(img), img_num])
    
    np.save('ml65-data/test_data.npy', np.array(testing_data, dtype=object))
    return testing_data

In [15]:
train_data = create_training_data()

100%|██████████| 25000/25000 [00:11<00:00, 2156.80it/s]


In [16]:
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression
from tensorflow.python.framework import ops
ops.reset_default_graph()

convnet = input_data(shape=[None, IMG_SIZE, IMG_SIZE, 1], name='input')

convnet = conv_2d(convnet, 32, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 64, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 32, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 64, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 32, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 64, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = fully_connected(convnet, 1024, activation='relu')
convnet = dropout(convnet, 0.8)

convnet = fully_connected(convnet, 2, activation='softmax')
convnet = regression(convnet, optimizer='adam', learning_rate=LR, 
                     loss='categorical_crossentropy', name='targets')

model = tflearn.DNN(convnet, tensorboard_dir='ml65-data/log')

In [24]:
if os.path.exists('ml65-data/{}.meta'.format(MODEL_NAME)):
    model.load('ml65-data/' + MODEL_NAME)
    print("Model loaded")

INFO:tensorflow:Restoring parameters from c:\Users\Aweso\our-code\MachineLearning\ml65-data\dogsvscat-0.001-6conv-basic.model
Model loaded


In [18]:
train = train_data[:-500]
test = train_data[-500:]

In [19]:
X = np.array([i[0] for i in train]).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
Y = [i[1] for i in train]

test_x = np.array([i[0] for i in train]).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
test_y = [i[1] for i in train]

In [20]:
model.fit({'input': X}, {'targets': Y}, n_epoch=5, validation_set=({'input': test_x}, {'targets': test_y}), snapshot_step=500, show_metric=True, run_id=MODEL_NAME)

Training Step: 1914  | total loss: 0.53596 | time: 2.259s
| Adam | epoch: 005 | loss: 0.53596 - acc: 0.7513 -- iter: 24448/24500
Training Step: 1915  | total loss: 0.52865 | time: 3.740s
| Adam | epoch: 005 | loss: 0.52865 - acc: 0.7558 | val_loss: 0.53250 - val_acc: 0.7328 -- iter: 24500/24500
--


In [21]:
model.save('ml65-data/' + MODEL_NAME)

INFO:tensorflow:c:\Users\Aweso\our-code\MachineLearning\ml65-data\dogsvscat-0.001-6conv-basic.model is not in all_model_checkpoint_paths. Manually adding it.
